In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation

In [2]:
conn = sqlite3.connect('./percentage.sqlite3')
c = conn.cursor()
per_data = pd.read_sql('SELECT * FROM percentage', con=conn)
c.close()

In [3]:
reverse_data = per_data[::-1]
reverse_data = reverse_data.reset_index(drop=True)
reverse_data

,date,press,percentage
0,2019-02-27,뉴시스,100
1,2019-02-27,연합뉴스,100
2,2019-02-27,조선비즈,100
3,2019-02-27,아시아경제,66
4,2019-02-27,이데일리,100
...,...,...,...
3231,2021-08-11,이데일리,72
3232,2021-08-11,파이낸셜뉴스,50
3233,2021-08-11,아시아경제,85
3234,2021-08-11,헤럴드경제,0


In [4]:
percentage = reverse_data['percentage'].values
percentage

array([100, 100, 100, ...,  85,   0,   0])

In [43]:
seq_len = 50      # 최근 50일의 예측점수를 관측치로 설정
sequence_length = seq_len + 1 # 하루가 지날 때마다 관측치 구간도 하루씩 이동

result = []
for index in range(len(percentage) - sequence_length):      # predictive_value: 언론사의 예측률 변수
    result.append(percentage[index: index + sequence_length])

# result

In [44]:
normalized_data = []
for window in result:
    for p in window:
      try:
        normalized_window = [(float(p) / float(window[0]) -1)]
        normalized_data.append(normalized_window)
      except:
        normalized_window = [0]
        normalized_data.append(normalized_window)

result = np.array(normalized_data)
# split train and test data

row = int(round(result.shape[0] * 0.9))
train = result[:row, :]
np.random.shuffle(train)

x_train = train[:, :-1]
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
y_train = train[:, -1]

# x_test = result[row:, :-1]
# x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
# y_test = result[row:, -1]

# x_train.shape, x_test.shape

[]


In [7]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(50, 1)))   # 관측치: 50일
model.add(LSTM(64, return_sequences=False))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='rmsprop')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50, 50)            10400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                29440     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 39,905
Trainable params: 39,905
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=10, epochs=50)

Epoch 1/50


ValueError: ignored